# 추천시스템

***

크게, **컨텐츠 기반 필터링 방식**과 **협업 필터링 방식**이 있다.

**딥러닝**, **하이브리드** 방식도 있다고 한다.

추천시스템 초기에는 컨텐츠 기반 방식을 많이 사용했는데 넷플릭스의 추천 시스템 이후로 latent factor와 같은 잠재적 요인을 반영하는 필터링 방식을 사용한다고 한다. 이 때는 **행렬 분해 기법**이 사용된다.

***

## 1. 컨텐츠 기반 필터링

#### 말그대로 사용자가 이용하는 아이템 혹은 서비스의 컨텐츠 내용을 기반으로 유사한 것을 추천해주는 방식이다. 상당히 간단한 접근으로 예를 들어 내가 먹방 영상 컨텐츠를 즐겨 볼 때 먹방과 유사한 영상을 추천해주는 식이다.

## 2. 협업 필터링

#### 협업필터링은 하나의 기준으로만 구현되는 것이 아닌 종합적인(=협업 느낌?) 접근으로 추천을 하는 방식이다. 예를 들어 사용자들이 구매했던 이력 정보 등의 행동 양식을 기반으로 추천을 한다. 하지만 희소성 행렬이 될 수 있는 문제점을 동시에 가지고 있다. 

#### 보통은 최근접 이웃 기반의 협업 방식을 사용하는데 이는 또 다시 **사용자 기반**과 **아이템 기반**으로 나뉘어 진다.

- 사용자 기반 : 구매 이력 정보가 유사한 사용자들이 이용한 아이템을 추천한다.
- 아이템 기반 : 해당 아이템을 이용한 사용자는 다음과 같은 아이템도 사용한다.
***

### 잠재 요인 협업 필터링

자매 요인 협업 필터링은 행렬 분해 기법을 기반하여 사용된다. SVD, NMF와 같은 차원 축소 기법으로 분해하며 **잠재요인**을 탐색한다.

즉, 사용자 - 아이템 행렬을 **사용자 * 잠재요인**, **아이템 * 잠재요인** 행렬로 분해를 할 수 있다. 하지만 실제로 잠재 요인이 구체적으로 어떤 것인지 알 수는 없다.

행렬 분해 방식은 **저장 공간 절약** 측면에서 큰 장점이 있다. 
***

## 컨텐츠 기반 필터링 구현

In [54]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
movies = pd.read_csv('tmdb_5000_movies.csv')

In [7]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [8]:
movies = movies[['id', 'genres', 'vote_average', 'vote_count', 'popularity', 'title', 'keywords', 'overview']]

In [10]:
movies.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


#### 평점과 관련하여 왜곡을 줄이기 위해 weighted rating(WR)을 제안하고 있다.

#### WR = (v / (v + m)) * R * (m / (v + m)) * C
- R = 개별 영화 평점
- v = 개별 영화 투표 횟수
- m = 250위 안에 들기 위해 필요한 최소 투표 수
- C = 전체 영화에 대한 평균 평점

In [12]:
m = movies['vote_count'].quantile(0.9)
movies = movies[movies.vote_count >= m]

In [14]:
C = movies['vote_average'].mean()

In [15]:
m, C

(1838.4000000000015, 6.9629937629937615)

In [22]:
def get_WR(data, m = m, C = C) :
    v = data['vote_count']
    R = data['vote_average']
    
    return (v / (v + m) * R * (m / (v +m) * C))

In [23]:
movies['WR'] = movies.apply(get_WR, axis = 1)

무엇을 컨텐츠로 잡을 것인가? 장르, 내용, 키워드로 잡아보자.

In [29]:
movies.genres = movies.genres.apply(literal_eval)
movies.keywords = movies.keywords.apply(literal_eval)

In [40]:
movies['genres'] = movies['genres'].apply(lambda x : [g['name'] for g in x])

In [41]:
movies['genres'] = movies['genres'].apply(lambda x : " ".join(x))

In [43]:
movies['keywords'] = movies['keywords'].apply(lambda x : [g['name'] for g in x])

In [44]:
movies['keywords'] = movies['keywords'].apply(lambda x : " ".join(x))

In [47]:
movies.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,WR
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",5.846873
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",9.893235
2,206647,Action Adventure Crime,6.3,4466,107.376788,Spectre,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,9.061660
3,49026,Action Crime Drama Thriller,7.6,9106,112.312950,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.395939
4,49529,Action Adventure Science Fiction,6.1,2124,43.926995,John Carter,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",10.563400


In [116]:
movies['content'] = movies['genres'].str.lower() + ' ' + movies['overview'].str.lower()

In [124]:
tfidf = TfidfVectorizer(max_features = 3000, ngram_range = (1, 3), lowercase = False, stop_words = 'english')

In [125]:
tfidf_vect = tfidf.fit_transform(movies['content'])

In [126]:
tfidf_vect.shape

(481, 3000)

In [127]:
content_sim = cosine_similarity(tfidf_vect, tfidf_vect)

In [128]:
content_sim = content_sim.argsort()[:, ::-1]

In [129]:
def get_rec_result(data, title, top = 10) :
    search_movie_index = data[data.title == title].index.values
    
    sim_index = content_sim[search_movie_index, :top + 1][0][1:]
    
    top10_movies = movies.iloc[sim_index][['title', 'WR']].sort_values(by = 'WR', ascending = False)
    return top10_movies

In [130]:
get_rec_result(movies, "The Dark Knight Rises")

,title,WR
1850,Scarface,13.177575
1596,Sicario,12.302371
1359,Batman,12.133003
3234,Nightcrawler,12.059284
1554,District 9,11.596446
790,American Sniper,10.640875
717,Jack Reacher,10.336268
119,Batman Begins,8.351912
65,The Dark Knight,6.576672
9,Batman v Superman: Dawn of Justice,6.536070
